Aight, right before closing the project, I encounter something interesting in this article - [On the Integration of Optical Flow and Action Recognition (GCPR 2018)](https://www.cvlibs.net/publications/Sevilla-Lara2018GCPR.pdf), and found i quiet fascinating, so I decided to do it, but a big question pops up in my mind; ***if I already fed 16 consecutive frames to my model to detect motion, why do I even need to do so?*** tune in for the answer at the end of this phase (by end phase i mean file '7.TwoStreams.ipynb' cuz this file is only for flow extraction)

### What is Optical Flow?

**Optical flow** is the pattern of apparent motion of objects between two consecutive frames in a video, caused by the movement of objects or the camera itself. It's a dense, pixel-wise motion field that estimates where each pixel in the first frame has moved to in the second.

---

### Why Extract Optical Flow?

Optical flow captures **temporal information**—what's changing over time, not just what's present in a single frame. This is crucial in:

* **Action recognition** (like your HMDB51 project): humans move, and we care about *how* they move.
* **Video understanding**: even a still person blinking or shifting can matter.
* **Scene dynamics**: camera pans, zooms, or object tracking all rely on motion cues.

Without optical flow, a model sees only *what* is in a scene. With flow, it sees *what’s happening*.

---

### X-flow and Y-flow — What Do They Show?

Once you compute optical flow, the result is usually split into **two components**:

* **x\_flow**  — the **horizontal displacement** of pixels between two frames (left ↔ right).
* **y\_flow**  — the **vertical displacement** (up ↕ down).

Imagine tracking a single pixel: If it moves 5 pixels to the right and 2 pixels up from one frame to the next, you'd have:
$x\_flow = +5, y\_flow = -2$

These values are usually stored as **grayscale images** where pixel intensity represents motion magnitude and direction.

---

### Optical Flow

At the heart of optical flow estimation is the **brightness constancy assumption**, which says:

$I(x,y,t) = I(x+\Delta x, y+\Delta y, t+\Delta t)$

This means a pixel’s brightness doesn't change as it moves; only its position does.
Using a Taylor expansion and dropping higher-order terms, we get the **optical flow constraint equation**:

$I_x \cdot v_x + I_y \cdot v_y + I_t = 0$

Where:

* $I_x$ and $I_y$ are the spatial gradients of the image
* $I_t$ is the temporal gradient (change over time)
* $v_x$ and $v_y$ are the optical flow components (what we're solving for)

In practice, algorithms like **Farneback**, **TV-L1**, or **Lucas-Kanade** are used to solve this under determined system by applying extra constraints. But I used Farneback.

### Why I Used Farneback Instead of TV-L1 for Optical Flow Extraction

In this project, I’ve opted to use the **Farneback method** to compute dense optical flow instead of the more accurate **TV-L1 algorithm** — and here’s why;

While **TV-L1** is known for producing high-quality, noise-resistant motion fields that are great for action recognition, it's also **computationally very expensive**. I ran a test and estimated that extracting flow for the entire HMDB51 dataset using TV-L1 would take over **86 hours** (you can see in the screen shot below) on my current setup — which simply isn’t practical for me right now. I’m not working with a high-end workstation or cluster, so resource constraints are a very real factor.

<img src="/Users/alesarabandi/Downloads/DEEPLEARING/project/Screenshot 2025-06-17 at 18.04.22.png" width="600">

Instead, I went with the **Farneback method**, which is **significantly faster** and allows me to complete the extraction in a reasonable amount of time, while still capturing the essential motion cues needed for comparing models.

---

### 📊 TV-L1 vs. Farneback — Comparison

| Feature              | TV-L1                           | Farneback                        |
|----------------------|----------------------------------|----------------------------------|
| Speed             | Very slow (hours/days)          | Fast (minutes/hours)            |
| Accuracy          | High (better for subtle motion) | Medium (less robust to noise)   |
| Compute Required  | High          | Low           |
| Output Quality    | Sharper, sparse-friendly flows  | Smoother, denser flows          |
| Use Case Fit      | Production / research-grade     | Prototyping / resource-limited  |

---




In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from glob import glob

### Looping through my dataset structure

In this bit, I'm walking through my dataset directory to collect **all video paths**, organised like this:  
`class_name/video_name`.

- I start with the top-level `frames` directory, where each subfolder is a different **action class**.
- Then I dig into each class folder to find the individual **video subfolders**.
- For each proper folder (not random files), I build a relative path like `brush_hair/video_001` and chuck it into my `all_videos` list.

Basically, I'm indexing my whole dataset so I can process the videos later.


In [ ]:
frames_dir = "/Users/alesarabandi/Downloads/DEEPLEARING/frames"  

all_videos = []

for class_name in sorted(os.listdir(frames_dir)):
    class_path = os.path.join(frames_dir, class_name)
    if not os.path.isdir(class_path):
        continue  # Skip non-folder files
    for video_name in sorted(os.listdir(class_path)):
        video_path = os.path.join(class_path, video_name)
        if os.path.isdir(video_path):
            rel_path = os.path.join(class_name, video_name)
            all_videos.append(rel_path)


### Extracting Optical Flow (Farneback style)

So here, I’m loading video frames and computing **Farneback optical flow** between consecutive frames — basically tracking motion across each clip. 🎥➡️🎥

#### What’s going on?

- First, I load all `.jpg` frames from a folder (thanks to `load_frames_from_folder`).
- Then, in `compute_and_save_farneback_flow`, I:
  - Read in all frame paths and make sure there’s at least 2 frames to work with.
  - Create a matching output folder for saving the flow results.
  - Loop over each frame pair (previous → current) and calculate motion using **Farneback’s method**.
  - I clip the flow to stay within `[-20, 20]` using the `bound` parameter — keeps things tidy and avoids weird spikes.
  - Then, I normalise the flow to `[0, 255]` for saving as image files.

#### What are `flow_x` and `flow_y`?

- `flow_x`: shows **horizontal** motion between frames (left ↔ right).
- `flow_y`: shows **vertical** motion (up ↕ down).

These are saved as separate greyscale images (`flow_x_0001.jpg`, `flow_y_0001.jpg`, etc.) so I can feed them into a model later.

#### The (basic) maths behind it

Optical flow estimates the displacement vector $\vec{d} = (u, v)$ at each pixel, assuming brightness constancy:

$$
I(x, y, t) \approx I(x + u, y + v, t + 1)
$$


Farneback’s method builds a polynomial expansion of the image signal in local windows, then computes flow from the deformation between those polynomials. It’s fast but not perfect.

All in all, this is a speed-friendly alternative to TV-L1 — not as precise, but waaay faster.

---


In [4]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image

def load_frames_from_folder(folder):
    """Loads all .jpg frames from a folder, sorted by filename."""
    frames = []
    for filename in sorted(os.listdir(folder)):
        if filename.endswith(".jpg"):
            frame = cv2.imread(os.path.join(folder, filename))
            if frame is not None:
                frames.append(frame)
    return frames

def compute_and_save_farneback_flow(video_folder, output_root, bound=20):
    frame_paths = sorted(glob(os.path.join(video_folder, '*.jpg')))
    if len(frame_paths) < 2:
        print(f"Skipped: Not enough frames in {video_folder}")
        return

    output_folder = video_folder.replace('/frames/frames', output_root)
    os.makedirs(output_folder, exist_ok=True)

    prev = cv2.imread(frame_paths[0])
    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

    for i in range(1, len(frame_paths)):
        curr = cv2.imread(frame_paths[i])
        curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)

        flow = cv2.calcOpticalFlowFarneback(
            prev_gray, curr_gray,
            None,
            pyr_scale=0.5, levels=3, winsize=15,
            iterations=3, poly_n=5, poly_sigma=1.2,
            flags=0
        )

        # Clip and normalise
        flow = np.clip(flow, -bound, bound)
        flow = ((flow + bound) * (255.0 / (2 * bound))).astype(np.uint8)

        # Split and save x and y
        flow_x, flow_y = flow[..., 0], flow[..., 1]
        cv2.imwrite(os.path.join(output_folder, f'flow_x_{i:04d}.jpg'), flow_x)
        cv2.imwrite(os.path.join(output_folder, f'flow_y_{i:04d}.jpg'), flow_y)

        prev_gray = curr_gray



### Kicking Off the Flow Extraction

this is where I actually **run the Farneback optical flow** extraction across my whole dataset. Let’s break it down:


- My `frames_dir` points to the root folder where each **class** has its own folder, and inside that are folders for each **video**.
- I define `output_root` for where I’ll save all the computed flow images.

#### Next:

- I grab all video directories using `glob` — so paths look like:  
  `frames_dir/class_name/video_name/*.jpg`.
- I then loop through every video folder using `tqdm` to get a progress bar.

For every folder:
- I run `compute_and_save_farneback_flow()` to generate optical flow.
- If anything breaks (e.g., weird folders, missing files), I catch the error and print a message so it doesn’t crash the whole loop.

This script just **batch processes** my entire dataset, one video at a time, and builds the `flow_x` and `flow_y` images for each clip.

In [ ]:
frames_dir = "/Users/alesarabandi/Downloads/DEEPLEARING/frames"  
output_root = "/Users/alesarabandi/Downloads/DEEPLEARING"

# Each video is under: frames_dir/class_name/video_name/*.jpg
video_dirs = sorted(glob(os.path.join(frames_dir, '*', '*')))  # class/video

print(f"Found {len(video_dirs)} videos.")

for video_folder in tqdm(video_dirs, desc="Computing Farneback Flow"):
    try:
        compute_and_save_farneback_flow(video_folder, output_root)
    except Exception as e:
        print(f"Failed for {video_folder}: {e}")


Found 6766 videos.


Computing Farneback Flow:   0%|          | 24/6766 [00:08<34:48,  3.23it/s]

Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_fr_goo_0
Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_1
Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_2
Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3


Computing Farneback Flow:  23%|██▎       | 1574/6766 [09:03<34:25,  2.51it/s] 

Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_avi_fencing_f_cm_np2_le_goo_0
Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_avi_fencing_u_cm_np2_ba_goo_1
Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_avi_fencing_u_cm_np2_fr_goo_2
Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_avi_fencing_u_cm_np2_fr_goo_3
Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_fencing_f_cm_np2_le_goo_0
Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_fencing_u_cm_np2_ba_goo_1
Skipped: Not enough frames in /Users/alesa

Computing Farneback Flow:  25%|██▍       | 1691/6766 [09:44<28:19,  2.99it/s]

Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/flic_flac/BHS___FlickFlack_[Tutorial]_flic_flac_f_cm_np1_le_med_0


Computing Farneback Flow:  69%|██████▉   | 4660/6766 [24:48<09:36,  3.65it/s]

Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/situp/Ab_Workout__(_6_pack_abs_)_[_ab_exercises_for_ripped_abs_]_situp_f_nm_np1_le_goo_0
Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/situp/Ab_Workout__(_6_pack_abs_)_[_ab_exercises_for_ripped_abs_]_situp_f_nm_np1_le_goo_1
Skipped: Not enough frames in /Users/alesarabandi/Downloads/DEEPLEARING/frames/situp/Ab_Workout__(_6_pack_abs_)_[_ab_exercises_for_ripped_abs_]_situp_f_nm_np1_le_goo_2


Computing Farneback Flow: 100%|██████████| 6766/6766 [36:30<00:00,  3.09it/s]


### Tidying Up the Flow Files

So after generating all the optical flow images, things got a bit messy in my dataset — flow files (`flow_x_*.jpg`, `flow_y_*.jpg`) were scattered right inside each video folder.

#### What I’m doing here:

- I loop through each class folder inside my `frames_root`.
- Then for each video folder within that class, I:
  1. Create a subfolder called `flows` .
  2. Move all the `flow_x_*.jpg` and `flow_y_*.jpg` files into that `flows` folder using `shutil.move()` .


In [ ]:
import os
import shutil
from tqdm import tqdm

frames_root = "/Users/alesarabandi/Downloads/DEEPLEARING/frames"  # my current mixed folder root

for class_name in tqdm(os.listdir(frames_root), desc="Organising flows"):
    class_path = os.path.join(frames_root, class_name)
    if not os.path.isdir(class_path):
        continue

    for video_name in os.listdir(class_path):
        video_path = os.path.join(class_path, video_name)
        if not os.path.isdir(video_path):
            continue

        # Create 'flows' subfolder inside each video folder
        flows_path = os.path.join(video_path, "flows")
        os.makedirs(flows_path, exist_ok=True)

        # Move flow images into the 'flows' folder
        for fname in os.listdir(video_path):
            if fname.startswith("flow_x_") or fname.startswith("flow_y_"):
                src = os.path.join(video_path, fname)
                dst = os.path.join(flows_path, fname)
                shutil.move(src, dst)


Organising flows: 100%|██████████| 52/52 [01:05<00:00,  1.26s/it]


### Removing Broken or Corrupted Video Folders

Some of the videos in my dataset were totally unusable — either missing frames, broken audio, or just a mess. So, I made a list of known bad apples and deleted them manually to keep my dataset clean and consistent. It is not a big deal, they're just 16 videos from 7k, so I prefer to delete them just to prevent this small issue transform to an avalanche later. 

#### What’s happening:

- I defined a list of **relative paths** to dodgy video folders that I want gone. 
- Then, for each one:
  - I build its **absolute path** from my `frames` root.
  - If the folder exists, I nuke it with `shutil.rmtree()`.
  - Otherwise, I just log that it’s already gone — no fuss.

This is essential for preventing crashes during training or flow extraction later down the line.


In [ ]:
import shutil
import os

#  root path
fr_root = "/Users/alesarabandi/Downloads/DEEPLEARING/frames"

# Relative paths from the correct root
relative_paths_to_delete = [
    "brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_fr_goo_0",
    "brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_1",
    "brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_2",
    "brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3",
    "fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_avi_fencing_f_cm_np2_le_goo_0",
    "fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_avi_fencing_u_cm_np2_ba_goo_1",
    "fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_avi_fencing_u_cm_np2_fr_goo_2",
    "fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_avi_fencing_u_cm_np2_fr_goo_3",
    "fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_fencing_f_cm_np2_le_goo_0",
    "fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_fencing_u_cm_np2_ba_goo_1",
    "fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_fencing_u_cm_np2_fr_goo_2",
    "fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_fencing_u_cm_np2_fr_goo_3",
    "flic_flac/BHS___FlickFlack_[Tutorial]_flic_flac_f_cm_np1_le_med_0",
    "situp/Ab_Workout__(_6_pack_abs_)_[_ab_exercises_for_ripped_abs_]_situp_f_nm_np1_le_goo_0",
    "situp/Ab_Workout__(_6_pack_abs_)_[_ab_exercises_for_ripped_abs_]_situp_f_nm_np1_le_goo_1",
    "situp/Ab_Workout__(_6_pack_abs_)_[_ab_exercises_for_ripped_abs_]_situp_f_nm_np1_le_goo_2"
]

# Now delete those folders
for rel_path in relative_paths_to_delete:
    abs_path = os.path.join(fr_root, rel_path)
    if os.path.exists(abs_path):
        shutil.rmtree(abs_path)
        print(f"✅ Deleted: {abs_path}")
    else:
        print(f"⚠️ Not found (already gone?): {abs_path}")


✅ Deleted: /Users/alesarabandi/Downloads/DEEPLEARING/frames/brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_fr_goo_0
✅ Deleted: /Users/alesarabandi/Downloads/DEEPLEARING/frames/brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_1
✅ Deleted: /Users/alesarabandi/Downloads/DEEPLEARING/frames/brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_2
✅ Deleted: /Users/alesarabandi/Downloads/DEEPLEARING/frames/brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_3
✅ Deleted: /Users/alesarabandi/Downloads/DEEPLEARING/frames/fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_avi_fencing_f_cm_np2_le_goo_0
✅ Deleted: /Users/alesarabandi/Downloads/DEEPLEARING/frames/fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_avi_fencing_u_cm_np2_ba_goo_1
✅ Deleted: /Users/alesarabandi/Downloads/DEEPLEARING/frames/fencing/Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_avi_fencing_u_cm_np2_fr_